# Test Run


In [2]:
# Load the autoreload extension
%load_ext autoreload

# Enable automatic reloading of modules
%autoreload 2

import sys
import os

sys.path.append(os.path.abspath(".."))


from tiny_graph.graph.base import Graph
from tiny_graph.constants import START, END

# Create a simple example graph
graph = Graph()


# Define some example actions
def start_action():
    print("Starting workflow")


def process_data():
    print("Processing data")


def validate():
    print("Validating results")


def end_action():
    print("Workflow complete")


# Add nodes to graph
graph.add_node("process", process_data)
graph.add_node("validate", validate)

# Add edges to create workflow
graph.add_edge(START, "process")
graph.add_edge("process", "validate")
graph.add_edge("validate", END)

# Visualize the graph
graph.visualize("example_workflow")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Concurrency testing


In [3]:
from threading import Lock
import threading
import time


class Counter:
    def __init__(self):
        self.count = 0
        self.lock = Lock()

    def increment_unsafe(self):
        # This method is not thread-safe
        current = self.count
        time.sleep(0.1)  # Simulate some work
        self.count = current + 1

    def increment_safe(self):
        # This method is thread-safe
        with self.lock:
            current = self.count
            time.sleep(0.1)  # Simulate some work
            self.count = current + 1


# Demo of race condition
def demo():
    counter = Counter()
    threads = []

    # Create 5 threads that increment unsafe
    for _ in range(5):
        t = threading.Thread(target=counter.increment_unsafe)
        threads.append(t)
        t.start()

    # Wait for all threads to complete
    for t in threads:
        t.join()

    print(f"Unsafe count should be 5, but is: {counter.count}")

    # Reset counter
    counter.count = 0
    threads = []

    # Create 5 threads that increment safe
    for _ in range(5):
        t = threading.Thread(target=counter.increment_safe)
        threads.append(t)
        t.start()

    # Wait for all threads to complete
    for t in threads:
        t.join()

    print(f"Safe count is correctly: {counter.count}")


demo()

Unsafe count should be 5, but is: 1
Safe count is correctly: 5


In [23]:
from typing import NamedTuple, Annotated
from pydantic import BaseModel


# Using NamedTuple
class PersonTuple(NamedTuple):
    name: str
    age: int
    city: str
    testing_list: Annotated[list[str], int]


# Using Pydantic
class PersonPydantic(BaseModel):
    name: str
    age: int
    city: str


# Create sample instances
person_tuple = PersonTuple(
    name="John Doe", age=30, city="New York", testing_list=["a", "b", "c"]
)
person_pydantic = PersonPydantic(name="Jane Doe", age=25, city="Boston")

print("NamedTuple Person:")
print(person_tuple)
print("\nPydantic Person:")
print(person_pydantic)

NamedTuple Person:
PersonTuple(name='John Doe', age=30, city='New York', testing_list=['a', 'b', 'c'])

Pydantic Person:
name='Jane Doe' age=25 city='Boston'


In [24]:
# ... existing PersonTuple and PersonPydantic definitions ...

# Get schema for NamedTuple
print("NamedTuple Schema:")
print(PersonTuple.__annotations__)  # Shows types of fields
print(PersonTuple._fields)  # Shows field names

# Get schema for Pydantic
print("\nPydantic Schema:")
print(PersonPydantic.model_json_schema())  # Full JSON schema
# For a simpler view of fields and types:
print("\nPydantic Fields:")
print(PersonPydantic.model_fields)
pydantic_schema = {
    field_name: field_info.annotation
    for field_name, field_info in PersonPydantic.model_fields.items()
}
print(pydantic_schema)

NamedTuple Schema:
{'name': <class 'str'>, 'age': <class 'int'>, 'city': <class 'str'>, 'testing_list': typing.Annotated[list[str], <class 'int'>]}
('name', 'age', 'city', 'testing_list')

Pydantic Schema:
{'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}, 'city': {'title': 'City', 'type': 'string'}}, 'required': ['name', 'age', 'city'], 'title': 'PersonPydantic', 'type': 'object'}

Pydantic Fields:
{'name': FieldInfo(annotation=str, required=True), 'age': FieldInfo(annotation=int, required=True), 'city': FieldInfo(annotation=str, required=True)}
{'name': <class 'str'>, 'age': <class 'int'>, 'city': <class 'str'>}


In [27]:
PersonTuple.__annotations__["testing_list"]

typing.Annotated[list[str], <class 'int'>]